In [3]:
# IMPORTS
import pandas as pd

from enron.src.common.const import *
from enron.src.data.import_data import *
from enron.src.features.data_cleaning import *

from manufacturing_company.src.common.const import *
from manufacturing_company.src.features.employee_activity import *


In [4]:
# DATA CLEANING
employees = load_employees()
emails = load_emails()

employees, emails = delete_nan(employees, emails)

employees.to_csv(ENRON_FILE_01_DELETE_NAN_EMPLOYEES, sep=';', index=True)
emails.to_csv(ENRON_FILE_01_DELETE_NAN_EMAILS, sep=';', index=False)

emails = remap_employee_ids(employees, emails)

employees.to_csv(ENRON_FILE_02_REMAP_EMPLOYEE_IDS_EMPLOYEES, sep=';', index=True)
emails.to_csv(ENRON_FILE_02_REMAP_EMPLOYEE_IDS_EMAILS, sep=';', index=False)

emails = delete_messages_sent_to_yourself(emails)

emails.to_csv(ENRON_FILE_03_DELETE_MESSAGE_SENT_TO_YOURSELF, sep=';', index=False)

emails = delete_messages_with_boundary_dates(emails)

emails.to_csv(ENRON_FILE_04_DELETE_MESSAGES_WITH_BOUNDARY_DATES, sep=';', index=False)

emails = emails.drop_duplicates([SENDER, RECIPIENT, EVENT_DATE])

emails.to_csv(ENRON_FILE_05_DELETE_DUPLICATE_MESSAGES, sep=';', index=False)


In [3]:
# MINIMUM EMPLOYEE ACTIVITY
emails[MONTH] = emails[EVENT_DATE].apply(lambda x: x.month)
emails[YEAR] = emails[EVENT_DATE].apply(lambda x: x.year)

for month in MONTHS:
    emails = remove_employees_below_minimum_activity(emails, month)
    emails.to_csv(ENRON_FILE_MINIMUM_ACTIVITY.format(month), sep=';', index=False)


In [5]:
# FLATTENING THE HIERARCHY
employees = pd.read_csv(ENRON_FILE_02_REMAP_EMPLOYEE_IDS_EMPLOYEES, sep=';')
employees[FLATTEN_POSITION] = -1

first_management_level = ['CEO', 'President', 'Vice President']
second_management_level = ['Director', 'Managing Director', 'Manager']
standard_employee = ['Employee', 'In House Lawyer', 'Trader']

employees.loc[employees[POSITION].isin(first_management_level), FLATTEN_POSITION] = 1
employees.loc[employees[POSITION].isin(second_management_level), FLATTEN_POSITION] = 2
employees.loc[employees[POSITION].isin(standard_employee), FLATTEN_POSITION] = 3

employees.to_csv(ENRON_FILE_FLATTEN_HIERARCHY, sep=';', index=False)

In [14]:
employees.groupby('flattenManagementLevels').agg('count')

,ID,email,name,ManagementLevel,details
flattenManagementLevels,,,,,
1,40,40,40,40,19
2,37,37,37,37,13
3,53,53,53,53,14
